In [ ]:
#| default_exp Train


In [ ]:
#| hide
from nbdev.showdoc import *

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/franfram/llmtools/blob/main/nbs/02_Train_vectorstore_index_with_Llamaindex.ipynb)


In [ ]:
# @title Install and import dependencies
# !pip install llama-index transformers accelerate sentence_transformers
from llama_index import ServiceContext, VectorStoreIndex, SimpleDirectoryReader, set_global_service_context
import transformers
from llama_index.llms import HuggingFaceLLM
import torch
import os

/home/franfram/llmtools/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set dummy OpenAI API key

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/franfram/llmtools/blob/main/nbs/02_Train_vectorstore_index_with_Llamaindex.ipynb)


In [ ]:
 # @title Set dummy OpenAI API key
 # @markdown Without it, we will run into an authentication error ('no API key set')
os.environ['OPENAI_API_KEY']='sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [ ]:
# @title Set embedding model
# @markdown Highest rated in [MTEB benchmark](https://huggingface.co/spaces/mteb/leaderboard) for document retrieval is thenlper/gte-large
model = 'thenlper/gte-large' # @param {type:'string'}
if not os.path.exists('gte-large'): 
  !git clone https://huggingface.co/thenlper/gte-large
  print("Cloning repo")
else:
  print("repo already exists")

#remove git repo to avoid issues
!rm -rf ./gte-large/.git


service_context = ServiceContext.from_defaults(
  embed_model="local:" + model
)
set_global_service_context(service_context)

repo already exists


No sentence-transformers model found with name /home/franfram/.cache/torch/sentence_transformers/thenlper_gte-large. Creating a new one with MEAN pooling.


In [ ]:
# @title Initialize empty index with the given context to use our local embedding model
index = VectorStoreIndex([], service_context=service_context)

In [ ]:
# @title Define recursive function to add documents to the index by travelling folders and subfolders


#| export
def recursive_folder_search(folder):
    # get a list of all subdirectories in the given folder
    subdirectories = [f.path for f in os.scandir(folder) if f.is_dir()]
    # if there are no subdirectories, run the code
    if len(subdirectories) == 0:
        f=0
        for files in os.listdir(folder):
          document = SimpleDirectoryReader(folder).load_data()[f]
          f+=1
          index.insert(document)
    else:
        # if there are subdirectories, recursively call this function on each subdirectory
        for subdirectory in subdirectories:
            recursive_folder_search(subdirectory)



hello

In [ ]:
# @title Train vectorstore index
recursive_folder_search("../dataset")

In [ ]:
# @title Save index to disk
index.storage_context.persist(persist_dir="indexes/")

In [ ]:
# @title Retrieve chunks
# @markdown Once finished, we can use the already loaded index to test and retrieve chunks
retriever = index.as_retriever()
searchTerm = 'search term' # @param {type:'string'}
nodes = retriever.retrieve(searchTerm)

print(nodes)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/franfram/llmtools/blob/main/nbs/02_Train_vectorstore_index_with_Llamaindex.ipynb)